# Image Classification: Detecting Pediatric Pneumonia

## Overview

## Business Understanding

## Data Understanding

## Data Preparation

In [1]:
#importing packages

import random
import numpy as np
import tensorflow as tf


import numpy as np
import pandas as pd 
import splitfolders
import matplotlib.pyplot as plt
import seaborn as sns
import itertools


from tensorflow import keras

import tensorflow as tf
from tensorflow.math import confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras import models, layers, optimizers, regularizers
from tensorflow.keras.models import load_model
from sklearn.dummy import DummyClassifier
from sklearn.metrics import recall_score, classification_report, precision_score
from sklearn.metrics import plot_confusion_matrix, confusion_matrix, ConfusionMatrixDisplay

from tensorflow.keras import metrics
from tensorflow.keras.wrappers import scikit_learn


import warnings
warnings.filterwarnings('ignore')
%matplotlib inline 

In [2]:
pip install split-folders

Note: you may need to restart the kernel to use updated packages.


In [3]:
#use the splitfolders package to split our image data into train, validation, and test sets.
#this is now commented out so that we don't create a new folder each time we run the notebook


#splitfolders.ratio("Data", output="Data_Split",
#    seed=42, ratio=(.64, .16, .2), group_prefix=None, move=True)

In [4]:
# Using ImageDataGenerator to rescale all images 
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [5]:
train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        'Data_Split/train',
        # All images will be resized to 150x150
        target_size=(150, 150),
        batch_size=4280,
        color_mode='grayscale',
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')


Found 3749 images belonging to 2 classes.


In [6]:
validation_generator = val_datagen.flow_from_directory('Data_Split/val',
                                                        target_size=(150, 150),
                                                        batch_size=978,
                                                        color_mode='grayscale',
                                                        class_mode='binary')
test_generator = test_datagen.flow_from_directory('Data_Split/test',
                                                  target_size=(150, 150),
                                                  batch_size=1173,
                                                  color_mode='grayscale',
                                                  class_mode='binary')

Found 936 images belonging to 2 classes.
Found 1107 images belonging to 2 classes.


In [7]:
#Creating the augumented data

aug_train_images = ImageDataGenerator(rotation_range=30, 
                                   width_shift_range=0.25, 
                                   height_shift_range=0.25, 
                                   shear_range=0.25, 
                                   zoom_range=0.25, 
                                   horizontal_flip=True,
                                   vertical_flip=True)

train_aug = aug_train_images.flow_from_directory('Data_Split/train',
                                                  target_size=(150, 150),
                                                  batch_size=3747,
                                                  color_mode='grayscale',
                                                  class_mode='binary')

Found 3749 images belonging to 2 classes.


In [ ]:
#getting images and labels for models
train_data, train_labels = next (train_generator)
test_data, test_labels = next (test_generator)
val_data, val_labels = next (validation_generator)

In [ ]:
#reshaping
train_data = train_data.reshape(train_data.shape[0], -1)
test_data = test_data.reshape(test_data.shape[0], -1)
val_data = val_data.reshape(val_data.shape[0], -1)
train_data.shape

In [ ]:
# Function to show confusion matrix 
##from sklearn
def plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

### Dummy Model
We will use a dummy model classifier as our baseline model. This model will predict the majority class. Since the majority class in our data is pneumonia, this model will predict all images are pneumonia.

In [ ]:
dummy_model =  DummyClassifier(strategy='most_frequent')
dummy_model.fit(train_data, train_labels)

In [ ]:
y_preds_dummy = dummy_model.predict(val_data)

In [ ]:
dummy_acc = dummy_model.score(val_data, val_labels)
dummy_rec = recall_score(val_labels,y_preds_dummy)
dummy_pre = precision_score(val_labels,y_preds_dummy)

print(f"Dummy Model accuracy: {dummy_acc}")
print(f"Dummy Model recall: {dummy_rec}")
print(f"Dummy Model precision: {dummy_pre}")

In [ ]:
y_pred = (dummy_model.predict(val_data) > 0.5).astype("int32")
cm = confusion_matrix(y_true= val_labels, y_pred=y_pred) 

In [ ]:
cm_labels = ['Normal','Pneumonia']
plot_confusion_matrix(cm=cm, classes=cm_labels, title='Confusion Matrix');

### Simple Model 1
Our first model will be a simple model - this will give us an idea of what improvements

In [ ]:
simple_model = models.Sequential()

In [ ]:
simple_model.add(layers.Dense(12, activation='relu', input_shape=(22500,)))
simple_model.add(layers.Dense(1, activation='sigmoid')) 

In [ ]:
simple_model.compile(optimizer='SGD',
                       loss='binary_crossentropy',
                       metrics=['accuracy', metrics.Precision(name='precision'), metrics.Recall(name='recall')])
simple_model.summary()

In [ ]:
simple_model_history = simple_model.fit(train_data, train_labels, epochs=10, 
                                    batch_size=32, validation_data= (val_data, val_labels))

In [ ]:

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2,figsize=(15,8))

loss = simple_model_history.history['loss']
accuracy = simple_model_history.history['accuracy']
precision = simple_model_history.history['precision']
recall = simple_model_history.history['recall']

validation_loss = simple_model_history.history['val_loss']
validation_accuracy = simple_model_history.history['val_accuracy']
validation_precision = simple_model_history.history['val_precision']
validation_recall = simple_model_history.history['val_recall']

sns.lineplot(simple_model_history.epoch, simple_model_history.history['loss'], y=loss, ax=ax1, label='loss')
sns.lineplot(simple_model_history.epoch, simple_model_history.history['val_loss'], y=loss, ax=ax1, label='val_loss')


sns.lineplot(simple_model_history.epoch, simple_model_history.history['accuracy'], y=accuracy, ax=ax2, label='accuracy')
sns.lineplot(simple_model_history.epoch, simple_model_history.history['val_accuracy'], y=accuracy, ax=ax2, label='val_accuracy')


sns.lineplot(simple_model_history.epoch, simple_model_history.history['precision'], y=precision, ax=ax3, label='precision')
sns.lineplot(simple_model_history.epoch, simple_model_history.history['val_precision'], y=precision, ax=ax3, label='val_precision')


sns.lineplot(simple_model_history.epoch, simple_model_history.history['recall'], y=recall, ax=ax4, label='recall')
sns.lineplot(simple_model_history.epoch, simple_model_history.history['val_recall'], y=recall, ax=ax4, label='val_recall')



In [ ]:
y_pred = (simple_model.predict(val_data) > 0.5).astype("int32")
cm = confusion_matrix(y_true= val_labels, y_pred=y_pred) 

In [ ]:
cm_labels = ['Normal','Pneumonia']
plot_confusion_matrix(cm=cm, classes=cm_labels, title='Confusion Matrix');

In [ ]:
results = simple_model.evaluate(val_data, val_labels)

In [ ]:
print(f"Model loss:  {results[0]}")
print(f"Model accuracy: {results[1]}")
print(f"Model precision: {results[2]}")
print(f"Model recall: {results[3]}")

### CNN Model 1
Our basic CNN model

In [ ]:
train_data, train_labels = next (train_generator)
test_data, test_labels = next (test_generator)
val_data, val_labels = next (validation_generator)

In [ ]:
cnn1_model = models.Sequential()
cnn1_model.add(layers.Conv2D(64, (4, 4), activation='relu',
                    input_shape=(150, 150, 1)))
cnn1_model.add(layers.MaxPooling2D((2, 2)))
cnn1_model.add(layers.Flatten())
cnn1_model.add(layers.Dense(16, activation='relu'))
cnn1_model.add(layers.Dense(1, activation='sigmoid'))

cnn1_model.compile(optimizer="adam",
                        loss='binary_crossentropy',
                        metrics=['accuracy', metrics.Precision(name='precision'), metrics.Recall(name='recall')])

cnn1_model.summary()

In [ ]:
cnn1_history = cnn1_model.fit(train_data,
               train_labels,
               batch_size=30,
               epochs=10,
               validation_data=(val_data, val_labels))

In [ ]:
cnn1_model.save('models/cnn1')

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2,figsize=(15,8))

sns.lineplot(cnn1_history.epoch, cnn1_history.history['loss'], y=loss, ax=ax1, label='loss')
sns.lineplot(cnn1_history.epoch, cnn1_history.history['val_loss'], y=loss, ax=ax1, label='val_loss')

sns.lineplot(cnn1_history.epoch, cnn1_history.history['accuracy'], y=accuracy, ax=ax2, label='accuracy')
sns.lineplot(cnn1_history.epoch, cnn1_history.history['val_accuracy'], y=accuracy, ax=ax2, label='val_accuracy')

sns.lineplot(cnn1_history.epoch, cnn1_history.history['precision'], y=precision, ax=ax3, label='precision')
sns.lineplot(cnn1_history.epoch, cnn1_history.history['val_precision'], y=precision, ax=ax3, label='val_precision')

sns.lineplot(cnn1_history.epoch, cnn1_history.history['recall'], y=recall, ax=ax4, label='recall')
sns.lineplot(cnn1_history.epoch, cnn1_history.history['val_recall'], y=recall, ax=ax4, label='val_recall');


In [ ]:
y_pred = (cnn1_model.predict(val_data) > 0.5).astype("int32")
cm = confusion_matrix(y_true= val_labels, y_pred=y_pred)  

In [ ]:
cm_labels = ['Normal','Pneumonia']
plot_confusion_matrix(cm=cm, classes=cm_labels, title='Confusion Matrix');

In [ ]:
results = cnn1_model.evaluate(validation_generator)

In [ ]:
print(f"Model loss:  {results[0]}")
print(f"Model accuracy: {results[1]}")
print(f"Model precision: {results[2]}")
print(f"Model recall: {results[3]}")

## CNN Model 2
description

In [ ]:
cnn2_model = models.Sequential()
cnn2_model.add(layers.Conv2D(64, (3, 3), activation='relu', 
                            input_shape=(150,150,1)))
cnn2_model.add(layers.MaxPooling2D((2,2)))
cnn2_model.add(layers.Conv2D(32, (3,3), activation='relu'))       
cnn2_model.add(layers.MaxPooling2D((2,2)))
cnn2_model.add(layers.Flatten())
cnn2_model.add(Dense(16, activation='relu'))
cnn2_model.add(Dense(1, activation='sigmoid')) 

cnn2_model.compile(optimizer="adam",
                        loss='binary_crossentropy',
                        metrics=['accuracy', metrics.Precision(name='precision'), metrics.Recall(name='recall')])

cnn2_model.summary()

In [ ]:
cnn2_history = cnn2_model.fit(train_data,
               train_labels,
               batch_size=50,
               epochs=10,
               validation_data=(val_data, val_labels))

In [ ]:
cnn2_model.save('models/cnn2')

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2,figsize=(15,8))

sns.lineplot(cnn2_history.epoch, cnn2_history.history['loss'], y=loss, ax=ax1, label='loss')
sns.lineplot(cnn2_history.epoch, cnn2_history.history['val_loss'], y=loss, ax=ax1, label='val_loss')

sns.lineplot(cnn2_history.epoch, cnn2_history.history['accuracy'], y=accuracy, ax=ax2, label='accuracy')
sns.lineplot(cnn2_history.epoch, cnn2_history.history['val_accuracy'], y=accuracy, ax=ax2, label='val_accuracy')

sns.lineplot(cnn2_history.epoch, cnn2_history.history['precision'], y=precision, ax=ax3, label='precision')
sns.lineplot(cnn2_history.epoch, cnn2_history.history['val_precision'], y=precision, ax=ax3, label='val_precision')

sns.lineplot(cnn2_history.epoch, cnn2_history.history['recall'], y=recall, ax=ax4, label='recall')
sns.lineplot(cnn2_history.epoch, cnn2_history.history['val_recall'], y=recall, ax=ax4, label='val_recall');

In [ ]:
y_pred = (cnn2_model.predict(val_data) > 0.5).astype("int32")
cm = confusion_matrix(y_true= val_labels, y_pred=y_pred)  

In [ ]:
cm_labels = ['Normal','Pneumonia']
plot_confusion_matrix(cm=cm, classes=cm_labels, title='Confusion Matrix');

In [ ]:
results = cnn2_model.evaluate(validation_generator)

In [ ]:
print(f"Model loss:  {results[0]}")
print(f"Model accuracy: {results[1]}")
print(f"Model precision: {results[2]}")
print(f"Model recall: {results[3]}")

### CNN Model 3

In [ ]:
cnn3_model = models.Sequential()
cnn3_model.add(layers.Conv2D(64, (4, 4), activation='relu',
                      input_shape=(150, 150, 1)))
cnn3_model.add(layers.MaxPooling2D((2, 2)))
cnn3_model.add(layers.Conv2D(32, (3, 3), activation='relu'))
cnn3_model.add(layers.MaxPooling2D((2,2)))
cnn3_model.add(layers.Flatten())
cnn3_model.add(layers.Dense(16, activation='relu'))
cnn3_model.add(layers.Dense(1, activation='sigmoid'))

cnn3_model.compile(optimizer="adam",
                          loss='binary_crossentropy',
                          metrics=['accuracy', metrics.Precision(name='precision'), metrics.Recall(name='recall')])

cnn3_model.summary()

In [ ]:
cnn3_history = cnn3_model.fit(train_data,
               train_labels,
               batch_size=32,
               epochs=10,
               validation_data=(val_data, val_labels))

In [ ]:
cnn3_model.save('models/cnn3')

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2,figsize=(15,8))

sns.lineplot(cnn3_history.epoch, cnn3_history.history['loss'], y=loss, ax=ax1, label='loss')
sns.lineplot(cnn3_history.epoch, cnn3_history.history['val_loss'], y=loss, ax=ax1, label='val_loss')

sns.lineplot(cnn3_history.epoch, cnn3_history.history['accuracy'], y=accuracy, ax=ax2, label='accuracy')
sns.lineplot(cnn3_history.epoch, cnn3_history.history['val_accuracy'], y=accuracy, ax=ax2, label='val_accuracy')

sns.lineplot(cnn3_history.epoch, cnn3_history.history['precision'], y=precision, ax=ax3, label='precision')
sns.lineplot(cnn3_history.epoch, cnn3_history.history['val_precision'], y=precision, ax=ax3, label='val_precision')

sns.lineplot(cnn3_history.epoch, cnn3_history.history['recall'], y=recall, ax=ax4, label='recall')
sns.lineplot(cnn3_history.epoch, cnn3_history.history['val_recall'], y=recall, ax=ax4, label='val_recall');

In [ ]:
y_pred = (cnn3_model.predict(val_data) > 0.5).astype("int32")
cm = confusion_matrix(y_true= val_labels, y_pred=y_pred)  

In [ ]:
cm_labels = ['Normal','Pneumonia']
plot_confusion_matrix(cm=cm, classes=cm_labels, title='Confusion Matrix');

In [ ]:
results = cnn3_model.evaluate(validation_generator)

In [ ]:
print(f"Model loss:  {results[0]}")
print(f"Model accuracy: {results[1]}")
print(f"Model precision: {results[2]}")
print(f"Model recall: {results[3]}")

### CNN Model 4

In [ ]:
cnn4_model = models.Sequential()
cnn4_model.add(layers.Conv2D(64, (4, 4), activation='relu',
                       input_shape=(150, 150, 1), kernel_regularizer=regularizers.l2(l=0.05)))
cnn4_model.add(layers.MaxPooling2D((2, 2)))
cnn4_model.add(layers.Conv2D(32, (3, 3), activation='relu', 
                                     kernel_regularizer=regularizers.l2(l=0.05)))
cnn4_model.add(layers.MaxPooling2D((2,2)))
cnn4_model.add(layers.Flatten())
cnn4_model.add(layers.Dense(16, activation='relu'))
cnn4_model.add(layers.Dense(1, activation='sigmoid'))

cnn4_model.compile(optimizer="adam",
                          loss='binary_crossentropy',
                          metrics=['accuracy', metrics.Precision(name='precision'), metrics.Recall(name='recall')])
cnn4_model.summary()

In [ ]:
cnn4_history = cnn4_model.fit(train_data,
              train_labels,
              batch_size=32,
              epochs=10,
              validation_data=(val_data, val_labels))

In [ ]:
cnn4_model.save('models/cnn4')

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2,figsize=(15,8))

sns.lineplot(cnn4_history.epoch, cnn4_history.history['loss'], y=loss, ax=ax1, label='loss')
sns.lineplot(cnn4_history.epoch, cnn4_history.history['val_loss'], y=loss, ax=ax1, label='val_loss')

sns.lineplot(cnn4_history.epoch, cnn4_history.history['accuracy'], y=accuracy, ax=ax2, label='accuracy')
sns.lineplot(cnn4_history.epoch, cnn4_history.history['val_accuracy'], y=accuracy, ax=ax2, label='val_accuracy')

sns.lineplot(cnn4_history.epoch, cnn4_history.history['precision'], y=precision, ax=ax3, label='precision')
sns.lineplot(cnn4_history.epoch, cnn4_history.history['val_precision'], y=precision, ax=ax3, label='val_precision')

sns.lineplot(cnn4_history.epoch, cnn4_history.history['recall'], y=recall, ax=ax4, label='recall')
sns.lineplot(cnn4_history.epoch, cnn4_history.history['val_recall'], y=recall, ax=ax4, label='val_recall');

In [ ]:
y_pred = (cnn4_model.predict(val_data) > 0.5).astype("int32")
cm = confusion_matrix(y_true= val_labels, y_pred=y_pred)  

In [ ]:
cm_labels = ['Normal','Pneumonia']
plot_confusion_matrix(cm=cm, classes=cm_labels, title='Confusion Matrix');

In [ ]:
results = cnn4_model.evaluate(validation_generator)

In [ ]:
print(f"Model loss:  {results[0]}")
print(f"Model accuracy: {results[1]}")
print(f"Model precision: {results[2]}")
print(f"Model recall: {results[3]}")

### CNN Model 5

In [ ]:
cnn5_model = models.Sequential()
cnn5_model.add(layers.Conv2D(64, (4, 4), activation='relu',
                       input_shape=(150, 150, 1), kernel_regularizer=regularizers.l2(l=0.05)))
cnn5_model.add(layers.MaxPooling2D((2, 2)))
cnn5_model.add(layers.Conv2D(32, (3, 3), activation='relu', 
                                     kernel_regularizer=regularizers.l2(l=0.05)))
cnn5_model.add(layers.MaxPooling2D((2,2)))
cnn5_model.add(layers.Flatten())
cnn5_model.add(layers.Dense(16, activation='relu'))
cnn5_model.add(layers.Dropout(0.5))
cnn5_model.add(layers.Dense(1, activation='sigmoid'))

cnn5_model.compile(optimizer="adam",
                          loss='binary_crossentropy',
                          metrics=['accuracy', metrics.Precision(name='precision'), metrics.Recall(name='recall')])

cnn5_model.summary()

In [ ]:
cnn5_history = cnn5_model.fit(train_data,
               train_labels,
               batch_size=32,
               epochs=10,
               validation_data=(val_data, val_labels))

In [ ]:
cnn5_model.save('models/cnn5')

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2,figsize=(15,8))

sns.lineplot(cnn5_history.epoch, cnn5_history.history['loss'], y=loss, ax=ax1, label='loss')
sns.lineplot(cnn5_history.epoch, cnn5_history.history['val_loss'], y=loss, ax=ax1, label='val_loss')

sns.lineplot(cnn5_history.epoch, cnn5_history.history['accuracy'], y=accuracy, ax=ax2, label='accuracy')
sns.lineplot(cnn5_history.epoch, cnn5_history.history['val_accuracy'], y=accuracy, ax=ax2, label='val_accuracy')

sns.lineplot(cnn5_history.epoch, cnn5_history.history['precision'], y=precision, ax=ax3, label='precision')
sns.lineplot(cnn5_history.epoch, cnn5_history.history['val_precision'], y=precision, ax=ax3, label='val_precision')

sns.lineplot(cnn5_history.epoch, cnn5_history.history['recall'], y=recall, ax=ax4, label='recall')
sns.lineplot(cnn5_history.epoch, cnn5_history.history['val_recall'], y=recall, ax=ax4, label='val_recall');

In [ ]:
y_pred = (cnn5_model.predict(val_data) > 0.5).astype("int32")
cm = confusion_matrix(y_true= val_labels, y_pred=y_pred)  

In [ ]:
cm_labels = ['Normal','Pneumonia']
plot_confusion_matrix(cm=cm, classes=cm_labels, title='Confusion Matrix');

In [ ]:
results = cnn5_model.evaluate(validation_generator)

In [ ]:
print(f"Model loss:  {results[0]}")
print(f"Model accuracy: {results[1]}")
print(f"Model precision: {results[2]}")
print(f"Model recall: {results[3]}")

### CNN Model 6

In [ ]:
cnn6_model = models.Sequential()
cnn6_model.add(layers.Conv2D(64, (4, 4), activation='relu',
                       input_shape=(150, 150, 1), kernel_regularizer=regularizers.l2(l=0.05)))
cnn6_model.add(layers.MaxPooling2D((2, 2)))
cnn6_model.add(layers.Conv2D(32, (3, 3), activation='relu', 
                                     kernel_regularizer=regularizers.l2(l=0.05)))
cnn6_model.add(layers.MaxPooling2D((2,2)))
cnn6_model.add(layers.Flatten())
cnn6_model.add(layers.Dense(16, activation='relu'))
cnn6_model.add(layers.Dropout(0.3))
cnn6_model.add(layers.Dense(1, activation='sigmoid'))

cnn6_model.compile(optimizer="adam",
                          loss='binary_crossentropy',
                          metrics=['accuracy', metrics.Precision(name='precision'), metrics.Recall(name='recall')])

cnn6_model.summary()

In [ ]:
cnn6_history = cnn6_model.fit(train_data,
               train_labels,
               batch_size=32,
               epochs=25,
               validation_data=(val_data, val_labels))

In [ ]:
cnn6_model.save('models/cnn6')

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2,figsize=(15,8))

sns.lineplot(cnn6_history.epoch, cnn6_history.history['loss'], y=loss, ax=ax1, label='loss')
sns.lineplot(cnn6_history.epoch, cnn6_history.history['val_loss'], y=loss, ax=ax1, label='val_loss')

sns.lineplot(cnn6_history.epoch, cnn6_history.history['accuracy'], y=accuracy, ax=ax2, label='accuracy')
sns.lineplot(cnn6_history.epoch, cnn6_history.history['val_accuracy'], y=accuracy, ax=ax2, label='val_accuracy')

sns.lineplot(cnn6_history.epoch, cnn6_history.history['precision'], y=precision, ax=ax3, label='precision')
sns.lineplot(cnn6_history.epoch, cnn6_history.history['val_precision'], y=precision, ax=ax3, label='val_precision')

sns.lineplot(cnn6_history.epoch, cnn6_history.history['recall'], y=recall, ax=ax4, label='recall')
sns.lineplot(cnn6_history.epoch, cnn6_history.history['val_recall'], y=recall, ax=ax4, label='val_recall');

In [ ]:
y_pred = (cnn6_model.predict(val_data) > 0.5).astype("int32")
cm = confusion_matrix(y_true= val_labels, y_pred=y_pred)  

In [ ]:
cm_labels = ['Normal','Pneumonia']
plot_confusion_matrix(cm=cm, classes=cm_labels, title='Confusion Matrix');

In [ ]:
results = cnn6_model.evaluate(validation_generator)

In [ ]:
print(f"Model loss:  {results[0]}")
print(f"Model accuracy: {results[1]}")
print(f"Model precision: {results[2]}")
print(f"Model recall: {results[3]}")

### CNN Model 7

In [ ]:
cnn7_model = models.Sequential()
cnn7_model.add(layers.Conv2D(64, (4, 4), activation='relu',
                       input_shape=(150, 150, 1), kernel_regularizer=regularizers.l2(l=0.025)))
cnn7_model.add(layers.MaxPooling2D((2, 2)))
cnn7_model.add(layers.Conv2D(32, (3, 3), activation='relu', 
                                     kernel_regularizer=regularizers.l2(l=0.025)))
cnn7_model.add(layers.MaxPooling2D((2,2)))
cnn7_model.add(layers.Flatten())
cnn7_model.add(layers.Dense(16, activation='relu'))
cnn7_model.add(layers.Dropout(0.3))
cnn7_model.add(layers.Dense(1, activation='sigmoid'))

cnn7_model.compile(optimizer="adam",
                          loss='binary_crossentropy',
                          metrics=['accuracy', metrics.Precision(name='precision'), metrics.Recall(name='recall')])
cnn7_model.summary()

In [ ]:
cnn7_history = cnn7_model.fit(train_data,
               train_labels,
               batch_size=32,
               epochs=25,
               validation_data=(val_data, val_labels))

In [ ]:
cnn7_model.save('models/cnn7')

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2,figsize=(15,8))

sns.lineplot(cnn7_history.epoch, cnn7_history.history['loss'], y=loss, ax=ax1, label='loss')
sns.lineplot(cnn7_history.epoch, cnn7_history.history['val_loss'], y=loss, ax=ax1, label='val_loss')

sns.lineplot(cnn7_history.epoch, cnn7_history.history['accuracy'], y=accuracy, ax=ax2, label='accuracy')
sns.lineplot(cnn7_history.epoch, cnn7_history.history['val_accuracy'], y=accuracy, ax=ax2, label='val_accuracy')

sns.lineplot(cnn7_history.epoch, cnn7_history.history['precision'], y=precision, ax=ax3, label='precision')
sns.lineplot(cnn7_history.epoch, cnn7_history.history['val_precision'], y=precision, ax=ax3, label='val_precision')

sns.lineplot(cnn7_history.epoch, cnn7_history.history['recall'], y=recall, ax=ax4, label='recall')
sns.lineplot(cnn7_history.epoch, cnn7_history.history['val_recall'], y=recall, ax=ax4, label='val_recall');

In [ ]:
y_pred = (cnn7_model.predict(val_data) > 0.5).astype("int32")
cm = confusion_matrix(y_true= val_labels, y_pred=y_pred)  

In [ ]:
cm_labels = ['Normal','Pneumonia']
plot_confusion_matrix(cm=cm, classes=cm_labels, title='Confusion Matrix');

In [ ]:
results = cnn7_model.evaluate(validation_generator)

In [ ]:
print(f"Model loss:  {results[0]}")
print(f"Model accuracy: {results[1]}")
print(f"Model precision: {results[2]}")
print(f"Model recall: {results[3]}")

### CNN Model 8

In [ ]:

cnn8_model = models.Sequential()
cnn8_model.add(layers.Conv2D(64, (4, 4), activation='relu',
                       input_shape=(150, 150, 1), kernel_regularizer=regularizers.l2(l=0.025)))
cnn8_model.add(layers.MaxPooling2D((3, 3)))
cnn8_model.add(layers.Conv2D(32, (3, 3), activation='relu', 
                                     kernel_regularizer=regularizers.l2(l=0.025)))
cnn8_model.add(layers.MaxPooling2D((3, 3)))
cnn8_model.add(layers.Flatten())
cnn8_model.add(layers.Dense(16, activation='relu'))
cnn8_model.add(layers.Dropout(0.3))
cnn8_model.add(layers.Dense(1, activation='sigmoid'))

cnn8_model.compile(optimizer="adam",
                          loss='binary_crossentropy',
                          metrics=['accuracy', metrics.Precision(name='precision'), metrics.Recall(name='recall')])

cnn8_model.summary()

In [ ]:
cnn8_history = cnn8_model.fit(train_data,
               train_labels,
               batch_size=32,
               epochs=25,
               validation_data=(val_data, val_labels))

In [ ]:
cnn8_model.save('models/cnn8')

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2,figsize=(15,8))

sns.lineplot(cnn8_history.epoch, cnn8_history.history['loss'], y=loss, ax=ax1, label='loss')
sns.lineplot(cnn8_history.epoch, cnn8_history.history['val_loss'], y=loss, ax=ax1, label='val_loss')

sns.lineplot(cnn8_history.epoch, cnn8_history.history['accuracy'], y=accuracy, ax=ax2, label='accuracy')
sns.lineplot(cnn8_history.epoch, cnn8_history.history['val_accuracy'], y=accuracy, ax=ax2, label='val_accuracy')

sns.lineplot(cnn8_history.epoch, cnn8_history.history['precision'], y=precision, ax=ax3, label='precision')
sns.lineplot(cnn8_history.epoch, cnn8_history.history['val_precision'], y=precision, ax=ax3, label='val_precision')

sns.lineplot(cnn8_history.epoch, cnn8_history.history['recall'], y=recall, ax=ax4, label='recall')
sns.lineplot(cnn8_history.epoch, cnn8_history.history['val_recall'], y=recall, ax=ax4, label='val_recall');

In [ ]:
y_pred = (cnn8_model.predict(val_data) > 0.5).astype("int32")
cm = confusion_matrix(y_true= val_labels, y_pred=y_pred)  

In [ ]:
cm_labels = ['Normal','Pneumonia']
plot_confusion_matrix(cm=cm, classes=cm_labels, title='Confusion Matrix');

In [ ]:
results = cnn8_model.evaluate(validation_generator)

In [ ]:
print(f"Model loss:  {results[0]}")
print(f"Model accuracy: {results[1]}")
print(f"Model precision: {results[2]}")
print(f"Model recall: {results[3]}")

### CNN Model 9

In [ ]:
cnn9_model = models.Sequential()
cnn9_model.add(layers.Conv2D(64, (4, 4), activation='relu',
                       input_shape=(150, 150, 1), kernel_regularizer=regularizers.l2(l=0.025)))
cnn9_model.add(layers.MaxPooling2D((3, 3)))
cnn9_model.add(layers.Conv2D(32, (3, 3), activation='relu', 
                                     kernel_regularizer=regularizers.l2(l=0.025)))
cnn9_model.add(layers.MaxPooling2D((3, 3)))
cnn9_model.add(layers.Conv2D(16, (2, 2), activation='relu', 
                                     kernel_regularizer=regularizers.l2(l=0.025)))
cnn9_model.add(layers.MaxPooling2D((3, 3)))
cnn9_model.add(layers.Flatten())
cnn9_model.add(layers.Dense(16, activation='relu'))
cnn9_model.add(layers.Dropout(0.3))
cnn9_model.add(layers.Dense(1, activation='sigmoid'))

cnn9_model.compile(optimizer="adam",
                          loss='binary_crossentropy',
                          metrics=['accuracy', metrics.Precision(name='precision'), metrics.Recall(name='recall')])

cnn9_model.summary()

In [ ]:
cnn9_history = cnn9_model.fit(train_data,
               train_labels,
               batch_size=32,
               epochs=25,
               validation_data=(val_data, val_labels))

In [ ]:
cnn9_model.save('models/cnn9')

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2,figsize=(15,8))

sns.lineplot(cnn9_history.epoch, cnn9_history.history['loss'], y=loss, ax=ax1, label='loss')
sns.lineplot(cnn9_history.epoch, cnn9_history.history['val_loss'], y=loss, ax=ax1, label='val_loss')

sns.lineplot(cnn9_history.epoch, cnn9_history.history['accuracy'], y=accuracy, ax=ax2, label='accuracy')
sns.lineplot(cnn9_history.epoch, cnn9_history.history['val_accuracy'], y=accuracy, ax=ax2, label='val_accuracy')

sns.lineplot(cnn9_history.epoch, cnn9_history.history['precision'], y=precision, ax=ax3, label='precision')
sns.lineplot(cnn9_history.epoch, cnn9_history.history['val_precision'], y=precision, ax=ax3, label='val_precision')

sns.lineplot(cnn9_history.epoch, cnn9_history.history['recall'], y=recall, ax=ax4, label='recall')
sns.lineplot(cnn9_history.epoch, cnn9_history.history['val_recall'], y=recall, ax=ax4, label='val_recall');

In [ ]:
y_pred = (cnn9_model.predict(val_data) > 0.5).astype("int32")
cm = confusion_matrix(y_true= val_labels, y_pred=y_pred)  

In [ ]:
cm_labels = ['Normal','Pneumonia']
plot_confusion_matrix(cm=cm, classes=cm_labels, title='Confusion Matrix');

In [ ]:
results = cnn9_model.evaluate(validation_generator)

In [ ]:
print(f"Model loss:  {results[0]}")
print(f"Model accuracy: {results[1]}")
print(f"Model precision: {results[2]}")
print(f"Model recall: {results[3]}")

### CNN Model 10

In [ ]:
cnn10_model = models.Sequential()
cnn10_model.add(layers.Conv2D(64, (4, 4), activation='relu',
                       input_shape=(150, 150, 1), kernel_regularizer=regularizers.l2(l=0.025)))
cnn10_model.add(layers.MaxPooling2D((3, 3)))
cnn10_model.add(layers.Conv2D(32, (3, 3), activation='relu', 
                                     kernel_regularizer=regularizers.l2(l=0.025)))
cnn10_model.add(layers.MaxPooling2D((3, 3)))
cnn10_model.add(layers.Conv2D(16, (2, 2), activation='relu', 
                                     kernel_regularizer=regularizers.l2(l=0.025)))
cnn10_model.add(layers.MaxPooling2D((3, 3)))
cnn10_model.add(layers.Flatten())
cnn10_model.add(layers.Dense(16, activation='relu'))
cnn10_model.add(layers.Dropout(0.3))
cnn10_model.add(layers.Dense(1, activation='sigmoid'))

cnn10_model.compile(optimizer="adam",
                          loss='binary_crossentropy',
                          metrics=['accuracy', metrics.Precision(name='precision'), metrics.Recall(name='recall')])

# Define the EarlyStopping object
early_stop = EarlyStopping(monitor='val_accuracy', min_delta=1e-8,
                           patience=10,
                           verbose=1,
                           mode='max')

cnn10_model.summary()

In [ ]:
cnn10_history = cnn10_model.fit(train_data,
               train_labels,
               batch_size=32,
               epochs=50,
               validation_data=(val_data, val_labels),
               callbacks=[early_stop])

In [ ]:
cnn10_model.save('models/cnn10')

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2,figsize=(15,8))

sns.lineplot(cnn10_history.epoch, cnn10_history.history['loss'], y=loss, ax=ax1, label='loss')
sns.lineplot(cnn10_history.epoch, cnn10_history.history['val_loss'], y=loss, ax=ax1, label='val_loss')

sns.lineplot(cnn10_history.epoch, cnn10_history.history['accuracy'], y=accuracy, ax=ax2, label='accuracy')
sns.lineplot(cnn10_history.epoch, cnn10_history.history['val_accuracy'], y=accuracy, ax=ax2, label='val_accuracy')

sns.lineplot(cnn10_history.epoch, cnn10_history.history['precision'], y=precision, ax=ax3, label='precision')
sns.lineplot(cnn10_history.epoch, cnn10_history.history['val_precision'], y=precision, ax=ax3, label='val_precision')

sns.lineplot(cnn10_history.epoch, cnn10_history.history['recall'], y=recall, ax=ax4, label='recall')
sns.lineplot(cnn10_history.epoch, cnn10_history.history['val_recall'], y=recall, ax=ax4, label='val_recall');


In [ ]:
y_pred = (cnn10_model.predict(val_data) > 0.5).astype("int32")
cm = confusion_matrix(y_true= val_labels, y_pred=y_pred) 
 

In [ ]:
cm_labels = ['Normal','Pneumonia']
plot_confusion_matrix(cm=cm, classes=cm_labels, title='Confusion Matrix');

In [ ]:
results = cnn10_model.evaluate(validation_generator)

In [ ]:
print(f"Model loss:  {results[0]}")
print(f"Model accuracy: {results[1]}")
print(f"Model precision: {results[2]}")
print(f"Model recall: {results[3]}")

### CNN Model 11

In [ ]:
cnn11_model = models.Sequential()
cnn11_model.add(layers.Conv2D(64, (4, 4), activation='relu',
                       input_shape=(150, 150, 1), kernel_regularizer=regularizers.l2(l=0.025)))
cnn11_model.add(layers.MaxPooling2D((3, 3)))
cnn11_model.add(layers.Conv2D(32, (3, 3), activation='relu', 
                                     kernel_regularizer=regularizers.l2(l=0.025)))
cnn11_model.add(layers.MaxPooling2D((3, 3)))
cnn11_model.add(layers.Flatten())
cnn11_model.add(layers.Dense(16, activation='relu'))
cnn11_model.add(layers.Dropout(0.3))
cnn11_model.add(layers.Dense(1, activation='sigmoid'))

# Define the EarlyStopping object
early_stop = EarlyStopping(monitor='val_loss', min_delta=1e-8,
                           patience=10,
                           verbose=1,
                           mode='min')

cnn11_model.compile(optimizer="adam",
                           loss='binary_crossentropy',
                           metrics=['accuracy', metrics.Precision(name='precision'), metrics.Recall(name='recall')])

cnn11_model.summary()

In [ ]:
cnn11_history = cnn11_model.fit(train_data,
               train_labels,
               batch_size=32,
               epochs=50,
               validation_data=(val_data, val_labels),
               callbacks=[early_stop])

In [ ]:
cnn11_model.save('models/cnn11')

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2,figsize=(15,8))

sns.lineplot(cnn11_history.epoch, cnn11_history.history['loss'], y=loss, ax=ax1, label='loss')
sns.lineplot(cnn11_history.epoch, cnn11_history.history['val_loss'], y=loss, ax=ax1, label='val_loss')

sns.lineplot(cnn11_history.epoch, cnn11_history.history['accuracy'], y=accuracy, ax=ax2, label='accuracy')
sns.lineplot(cnn11_history.epoch, cnn11_history.history['val_accuracy'], y=accuracy, ax=ax2, label='val_accuracy')

sns.lineplot(cnn11_history.epoch, cnn11_history.history['precision'], y=precision, ax=ax3, label='precision')
sns.lineplot(cnn11_history.epoch, cnn11_history.history['val_precision'], y=precision, ax=ax3, label='val_precision')

sns.lineplot(cnn11_history.epoch, cnn11_history.history['recall'], y=recall, ax=ax4, label='recall')
sns.lineplot(cnn11_history.epoch, cnn11_history.history['val_recall'], y=recall, ax=ax4, label='val_recall');


In [ ]:
y_pred = (cnn11_model.predict(val_data) > 0.5).astype("int32")
cm = confusion_matrix(y_true= val_labels, y_pred=y_pred) 


In [ ]:
cm_labels = ['Normal','Pneumonia']
plot_confusion_matrix(cm=cm, classes=cm_labels, title='Confusion Matrix');

In [ ]:
results = cnn11_model.evaluate(validation_generator)

In [ ]:
print(f"Model loss:  {results[0]}")
print(f"Model accuracy: {results[1]}")
print(f"Model precision: {results[2]}")
print(f"Model recall: {results[3]}")

### CNN Model 12

In [ ]:
cnn12_model = models.Sequential()
cnn12_model.add(layers.Conv2D(64, (4, 4), activation='relu',
                       input_shape=(150, 150, 1), kernel_regularizer=regularizers.l2(l=0.025)))
cnn12_model.add(layers.MaxPooling2D((3, 3)))
cnn12_model.add(layers.Conv2D(32, (3, 3), activation='relu', 
                                     kernel_regularizer=regularizers.l2(l=0.025)))
cnn12_model.add(layers.MaxPooling2D((3, 3)))
cnn12_model.add(layers.Flatten())
cnn12_model.add(layers.Dense(16, activation='relu'))
cnn12_model.add(layers.Dropout(0.3))
cnn12_model.add(layers.Dense(1, activation='sigmoid'))

# Define the EarlyStopping object
early_stop = EarlyStopping(monitor='val_loss', min_delta=1e-8,
                           patience=10,
                           verbose=1,
                           mode='min')

cnn12_model.compile(optimizer="adam",
                           loss='binary_crossentropy',
                           metrics=['accuracy', metrics.Precision(name='precision', thresholds=0.3), metrics.Recall(name='recall', thresholds=0.3)]) 

cnn12_model.summary()

In [ ]:
cnn12_history = cnn12_model.fit(train_data,
               train_labels,
               batch_size=32,
               epochs=30,
               validation_data=(val_data, val_labels),
               callbacks=[early_stop])

In [ ]:
cnn12_model.save('models/cnn12')

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2,figsize=(15,8))

sns.lineplot(cnn12_history.epoch, cnn12_history.history['loss'], y=loss, ax=ax1, label='loss')
sns.lineplot(cnn12_history.epoch, cnn12_history.history['val_loss'], y=loss, ax=ax1, label='val_loss')

sns.lineplot(cnn12_history.epoch, cnn12_history.history['accuracy'], y=accuracy, ax=ax2, label='accuracy')
sns.lineplot(cnn12_history.epoch, cnn12_history.history['val_accuracy'], y=accuracy, ax=ax2, label='val_accuracy')

sns.lineplot(cnn12_history.epoch, cnn12_history.history['precision'], y=precision, ax=ax3, label='precision')
sns.lineplot(cnn12_history.epoch, cnn12_history.history['val_precision'], y=precision, ax=ax3, label='val_precision')

sns.lineplot(cnn12_history.epoch, cnn12_history.history['recall'], y=recall, ax=ax4, label='recall')
sns.lineplot(cnn12_history.epoch, cnn12_history.history['val_recall'], y=recall, ax=ax4, label='val_recall');


In [ ]:
y_pred = (cnn12_model.predict(val_data) > 0.5).astype("int32")
cm = confusion_matrix(y_true= val_labels, y_pred=y_pred) 


In [ ]:
cm_labels = ['Normal','Pneumonia']
plot_confusion_matrix(cm=cm, classes=cm_labels, title='Confusion Matrix');

In [ ]:
results = cnn12_model.evaluate(validation_generator)

In [ ]:
print(f"Model loss:  {results[0]}")
print(f"Model accuracy: {results[1]}")
print(f"Model precision: {results[2]}")
print(f"Model recall: {results[3]}")